In [1]:
%reload_ext autotime
import pandas as pd
import geopandas as gpd
import requests
import requests_cache
requests_cache.install_cache('cache')
import streetview
import os
from tqdm.auto import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import dotenv
dotenv.load_dotenv()

True

In [2]:
df = pd.read_csv("example_GPS_data.csv")
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")
df

,pid,n,time,anxiousness,latitude,longitude,geometry
0,P20001,1,2023-04-25T02:51:42Z,0,-36.924795,174.738044,POINT (174.73804 -36.92479)
1,P20001,2,2023-04-25T08:43:13Z,0,-36.924801,174.738076,POINT (174.73808 -36.9248)
2,P20001,3,2023-04-26T02:22:26Z,0,-36.924783,174.738071,POINT (174.73807 -36.92478)
3,P20001,4,2023-04-26T09:00:53Z,0,-36.924797,174.738123,POINT (174.73812 -36.9248)
4,P20001,5,2023-04-26T20:36:19Z,3,-36.924771,174.738084,POINT (174.73808 -36.92477)
...,...,...,...,...,...,...,...
1426,P20721,1,2024-05-05T02:00:52Z,1,-36.893455,174.728262,POINT (174.72826 -36.89346)
1427,P20721,2,2024-05-05T07:36:22Z,1,-36.893483,174.728253,POINT (174.72825 -36.89348)
1428,P20721,3,2024-05-05T23:06:27Z,2,-36.845252,174.759951,POINT (174.75995 -36.84525)
1429,P20721,4,2024-05-06T07:04:57Z,0,-36.845165,174.759885,POINT (174.75988 -36.84516)


In [ ]:
results = []
for row in tqdm(df.itertuples(), total=len(df)):
    panoids = streetview.panoids(lat=row.latitude, lon=row.longitude)
    # Filter to recent panoramas
    panoids = [p for p in panoids if p.get("year", 2024) > 2023]
    if panoids:
        pano = panoids[0]
        panoid = pano["panoid"]
        if not pano.get("year"):
            # If the year is not available, get it from the metadata
            year, month = requests.get(f"https://maps.googleapis.com/maps/api/streetview/metadata?pano={panoid}&key={os.getenv('API_KEY')}").json()["date"].split("-")
            print(panoid, year, month)
            pano["year"] = year
            pano["month"] = month
        filename = f"panoramas/{panoid}.png"
        if not os.path.isfile(filename):
            try:
                panorama = streetview.download_panorama_v3(panoid, zoom=2, disp=False)
                os.makedirs("panoramas", exist_ok=True)
                Image.fromarray(panorama).save(filename)
            except Exception as e:
                print(e)
                display(row)
        row = row._asdict()
        row["panoid"] = panoid
        row["panolat"] = pano["lat"]
        row["panolon"] = pano["lon"]
        row["panoyear"] = pano["year"]
        row["panomonth"] = pano["month"]
        results.append(row)
        if row["Index"] % 10 == 0:
            pd.DataFrame(results).to_csv("results.csv", index=False)
pd.DataFrame(results).to_csv("results.csv", index=False)

  0%|          | 0/1431 [00:00<?, ?it/s]

QEpZV7bnO2mBfp0weMUKEg 2012 04
QEpZV7bnO2mBfp0weMUKEg 2012 04
QEpZV7bnO2mBfp0weMUKEg 2012 04
QEpZV7bnO2mBfp0weMUKEg 2012 04
QEpZV7bnO2mBfp0weMUKEg 2012 04
QEpZV7bnO2mBfp0weMUKEg 2012 04
QEpZV7bnO2mBfp0weMUKEg 2012 04
QEpZV7bnO2mBfp0weMUKEg 2012 04
QEpZV7bnO2mBfp0weMUKEg 2012 04
QEpZV7bnO2mBfp0weMUKEg 2012 04
pUw8PmVPYZBTW26mq2DAfw 2012 04
QEpZV7bnO2mBfp0weMUKEg 2012 04
QEpZV7bnO2mBfp0weMUKEg 2012 04
NQi8eh4MBHprJGCpl9t1EQ 2023 11
0dU2CZ_GXhuhodJJbXVHSA 2017 02
IUzaXGEkzyOV-oEgTQaHhA 2023 01
uPRCtL-OvwI3f_lXNBZtdw 2023 01
BoCOn1VpFGrbXlyX3EKZ6g 2022 08
BoCOn1VpFGrbXlyX3EKZ6g 2022 08
tbfXbYFHITDw8p7vCFU3KA 2022 08
BoCOn1VpFGrbXlyX3EKZ6g 2022 08
BoCOn1VpFGrbXlyX3EKZ6g 2022 08
BoCOn1VpFGrbXlyX3EKZ6g 2022 08
BoCOn1VpFGrbXlyX3EKZ6g 2022 08
u8H0X8VyWl2lDVkS6Ya_-A 2022 08
tbfXbYFHITDw8p7vCFU3KA 2022 08
BoCOn1VpFGrbXlyX3EKZ6g 2022 08
BoCOn1VpFGrbXlyX3EKZ6g 2022 08
E636s1bSOcFecsxRZBhe-A 2022 06
E636s1bSOcFecsxRZBhe-A 2022 06
E636s1bSOcFecsxRZBhe-A 2022 06
E636s1bSOcFecsxRZBhe-A 2022 06
E636s1bS